In [41]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import glob
import logging
import os
import shutil
import sys
import tempfile

import catalyst.dl
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
from monai.config import print_config
from monai.data import Dataset, create_test_image_3d, list_data_collate, decollate_batch
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.networks.nets import UNet
from monai.transforms import (
    Activations,
    AsChannelFirstd,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
    EnsureTyped,
    EnsureType,
)
from monai.utils import first

import torch

print_config()

MONAI version: 0.10.dev2228
Numpy version: 1.23.1
Pytorch version: 1.12.0+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: b287dfc6a7c397523ba67e10a0436099fb0a44f7
MONAI __file__: C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\venv\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 4.0.1
scikit-image version: 0.19.3
Pillow version: 9.2.0
Tensorboard version: 2.9.1
gdown version: 4.5.1
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.64.0
lmdb version: 1.3.0
psutil version: 5.9.1
pandas version: 1.4.3
einops version: 0.4.1
transformers version: 4.20.1
mlflow version: 1.27.0
pynrrd version: 0.4.3

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [42]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
root_dir = 'monai_unet/'

In [35]:
import os
print(os.getcwd())
for i in range(40):
    im, seg = create_test_image_3d(
        128, 128, 128, num_seg_classes=1, channel_dim=-1
    )

    n = nib.Nifti1Image(im, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"img{i}.nii.gz"))

    n = nib.Nifti1Image(seg, np.eye(4))
    nib.save(n, os.path.join(root_dir, f"seg{i}.nii.gz"))

images = sorted(glob.glob(os.path.join(root_dir, "img*.nii.gz")))
segs = sorted(glob.glob(os.path.join(root_dir, "seg*.nii.gz")))

print(images)
print(segs)

C:\Users\sathy\OneDrive\Documents\GitHub\Multi-Modality-Abdominal-Multi-Organ-Segmentation-Challenge-2022\monai_training
['monai_unet\\img0.nii.gz', 'monai_unet\\img1.nii.gz', 'monai_unet\\img10.nii.gz', 'monai_unet\\img11.nii.gz', 'monai_unet\\img12.nii.gz', 'monai_unet\\img13.nii.gz', 'monai_unet\\img14.nii.gz', 'monai_unet\\img15.nii.gz', 'monai_unet\\img16.nii.gz', 'monai_unet\\img17.nii.gz', 'monai_unet\\img18.nii.gz', 'monai_unet\\img19.nii.gz', 'monai_unet\\img2.nii.gz', 'monai_unet\\img20.nii.gz', 'monai_unet\\img21.nii.gz', 'monai_unet\\img22.nii.gz', 'monai_unet\\img23.nii.gz', 'monai_unet\\img24.nii.gz', 'monai_unet\\img25.nii.gz', 'monai_unet\\img26.nii.gz', 'monai_unet\\img27.nii.gz', 'monai_unet\\img28.nii.gz', 'monai_unet\\img29.nii.gz', 'monai_unet\\img3.nii.gz', 'monai_unet\\img30.nii.gz', 'monai_unet\\img31.nii.gz', 'monai_unet\\img32.nii.gz', 'monai_unet\\img33.nii.gz', 'monai_unet\\img34.nii.gz', 'monai_unet\\img35.nii.gz', 'monai_unet\\img36.nii.gz', 'monai_unet\\i

In [36]:
imgs = sorted(glob.glob(os.path.join("imagesTr/*.nii.gz")))
segmentations = sorted(glob.glob(os.path.join("labelsTr/*.nii.gz")))
print(imgs)
print(segmentations)

['imagesTr\\amos_0001.nii.gz', 'imagesTr\\amos_0004.nii.gz', 'imagesTr\\amos_0005.nii.gz', 'imagesTr\\amos_0006.nii.gz', 'imagesTr\\amos_0007.nii.gz', 'imagesTr\\amos_0009.nii.gz', 'imagesTr\\amos_0010.nii.gz', 'imagesTr\\amos_0011.nii.gz', 'imagesTr\\amos_0014.nii.gz', 'imagesTr\\amos_0015.nii.gz', 'imagesTr\\amos_0016.nii.gz', 'imagesTr\\amos_0017.nii.gz', 'imagesTr\\amos_0019.nii.gz', 'imagesTr\\amos_0021.nii.gz', 'imagesTr\\amos_0023.nii.gz', 'imagesTr\\amos_0024.nii.gz', 'imagesTr\\amos_0025.nii.gz', 'imagesTr\\amos_0027.nii.gz', 'imagesTr\\amos_0030.nii.gz', 'imagesTr\\amos_0033.nii.gz', 'imagesTr\\amos_0035.nii.gz', 'imagesTr\\amos_0036.nii.gz', 'imagesTr\\amos_0038.nii.gz', 'imagesTr\\amos_0042.nii.gz', 'imagesTr\\amos_0043.nii.gz', 'imagesTr\\amos_0044.nii.gz', 'imagesTr\\amos_0045.nii.gz', 'imagesTr\\amos_0047.nii.gz', 'imagesTr\\amos_0048.nii.gz', 'imagesTr\\amos_0049.nii.gz', 'imagesTr\\amos_0050.nii.gz', 'imagesTr\\amos_0052.nii.gz', 'imagesTr\\amos_0054.nii.gz', 'imagesTr

In [45]:
train_files = [
    {"image": img, "label": seg} for img, seg in zip(imgs[:20], segmentations[:20])
]
val_files = [
    {"image": img, "label": seg} for img, seg in zip(imgs[-20:], segmentations[-20:])
]

In [48]:
# define transforms for image and segmentation
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        AsChannelFirstd(keys=["image", "label"], channel_dim=-1),
        ScaleIntensityd(keys=["image", "label"]),
        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=[96, 96, 96],
            pos=1,
            neg=1,
            num_samples=4,
        ),
        RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
        EnsureTyped(keys=["image", "label"]),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        AsChannelFirstd(keys=["image", "label"], channel_dim=-1),
        ScaleIntensityd(keys=["image", "label"]),
        EnsureTyped(keys=["image", "label"]),
    ]
)

# define dataset, data loader
check_ds = Dataset(data=train_files, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
check_loader = torch.utils.data.DataLoader(
    check_ds, batch_size=2, num_workers=4, collate_fn=list_data_collate
)
check_data = first(check_loader)
print(check_data["image"].shape, check_data["label"].shape)

# create a training data loader
train_ds = Dataset(data=train_files, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
train_loader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=list_data_collate,
    pin_memory=torch.cuda.is_available(),
)
# create a validation data loader
val_ds = Dataset(data=val_files, transform=val_transforms)
val_loader = torch.utils.data.DataLoader(
    val_ds, batch_size=1, num_workers=4, collate_fn=list_data_collate
)

RuntimeError: DataLoader worker (pid(s) 2732, 4768, 22444, 25852) exited unexpectedly